In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
#parameters
vocab_size = 10000
maxlen = 200
embed_dim =32
num_heads = 2
ff_dim = 32

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
x_train = pad_sequences(x_train,maxlen=maxlen)
x_test = pad_sequences(x_test,maxlen=maxlen)

In [4]:
inputs = layers.Input(shape=(maxlen,))

In [5]:
embedding_layer = layers.Embedding(vocab_size,embed_dim)(inputs)

In [6]:
positions = tf.range(start=0, limit=maxlen, delta=1)
position_embeddings = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)(positions)
x = embedding_layer + position_embeddings

In [7]:
attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)

In [8]:
x = layers.LayerNormalization()(x+attention)

In [10]:
# feed forward network - thinking layer or processing brain
ffn = models.Sequential(
    [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
)

- currently each word is a vector for classification we need one sumarised vector 
- pooling will create a summary of all sentences 

In [11]:
# pooling layer 
x = layers.GlobalAveragePooling1D()(x)

In [12]:
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

In [13]:
model = models.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", 
            loss="binary_crossentropy", 
            metrics=["accuracy"])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 200, 32)   │    320,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 200, 32)   │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 200, 32)   │      8,416 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 200, 32)   │          0 │ add[0][0],        │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 200, 32)   │         64 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 32)        │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 20)        │        660 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         21 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 329,161 (1.26 MB)

 Trainable params: 329,161 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.fit(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    epochs=5,
    batch_size=64,
)

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 43s 111ms/step - accuracy: 0.8982 - loss: 0.2493 - val_accuracy: 0.8738 - val_loss: 0.2986
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 42s 107ms/step - accuracy: 0.9324 - loss: 0.1796 - val_accuracy: 0.8666 - val_loss: 0.3308
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 42s 108ms/step - accuracy: 0.9537 - loss: 0.1374 - val_accuracy: 0.8564 - val_loss: 0.3812
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 37s 95ms/step - accuracy: 0.9628 - loss: 0.1101 - val_accuracy: 0.8506 - val_loss: 0.4269
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 36s 92ms/step - accuracy: 0.9740 - loss: 0.0877 - val_accuracy: 0.8386 - val_loss: 0.5402
